# We now combine FLAIR with LVR

#### Helper functions

In [33]:
from web3 import Web3
import time
import requests
from datetime import datetime

# Initialize a Web3 instance
w3 = Web3(Web3.HTTPProvider('https://eth-mainnet.g.alchemy.com/v2/OV2Uqs1tC70MAtl88BcFArffiXobkx6T'))

# get timestamp for a given block
def get_timestamp(block_number):
    block = w3.eth.get_block(block_number)
    return block.timestamp * 1000

# get timestamp for current time
def get_current_timestamp():
    return int(time.time()) * 1000

# get block from timestamp using llama fi endpoint https://coins.llama.fi/block/ethereum/{timestamp}

def get_block_from_timestamp(timestamp):
    url = f"https://coins.llama.fi/block/ethereum/{timestamp}"
    try:
        response = requests.get(url)
        data = response.json()
        block_number = data["height"]
        return block_number
    except requests.exceptions.RequestException as e:
        print("Error occurred:", e)
        return None
    
def get_date(timestamp):
    return datetime.fromtimestamp(timestamp/1000).strftime('%Y-%m-%d')   

def get_eth_price():
    url = "https://api.coingecko.com/api/v3/simple/price"
    params = {
        "ids": "ethereum",
        "vs_currencies": "usd"
    }

    try:
        response = requests.get(url, params=params)
        data = response.json()
        eth_price = data["ethereum"]["usd"]
        return eth_price
    except requests.exceptions.RequestException as e:
        print("Error occurred:", e)
        return None
    
def get_coin_historical(timestamp,address):
    url = f"https://coins.llama.fi/prices/historical/{timestamp}/ethereum:{address}?searchWidth=4h"
    try:
        response = requests.get(url)
        data = response.json()
        return data['coins'][f'ethereum:{address}']['price']
    except requests.exceptions.RequestException as e:
        print("Error occurred:", e)
        return None

### Google BigQuery for Eth Block Timestamps

In [29]:
from google.cloud import bigquery
import pandas as pd

def get_block_timestamps(start_block, end_block):
    # Create a BigQuery client
    client = bigquery.Client(project="range-eth-research")

    # Define your query with a parameterized range of block numbers
    query = """
        SELECT number, timestamp
        FROM `bigquery-public-data.crypto_ethereum.blocks`
        WHERE number >= @start_block AND number < @end_block
    """

    # Create a query job configuration
    job_config = bigquery.QueryJobConfig()
    job_config.query_parameters = [
        bigquery.ScalarQueryParameter("start_block", "INT64", start_block),
        bigquery.ScalarQueryParameter("end_block", "INT64", end_block)
    ]

    # Execute the query with the job configuration
    query_job = client.query(query, job_config=job_config)

    # Fetch the results
    results = query_job.result()

    # Convert the results to a DataFrame
    df = pd.DataFrame(columns=["Block", "Timestamp"])
    for row in results:
        # Access the block number and timestamp values
        block_number = row["number"]
        timestamp = row["timestamp"].timestamp()
        df = df._append({"Block": int(block_number), "Timestamp": int(timestamp)}, ignore_index=True)

    return df

# Example usage: Get block timestamps from block number 17400393 to 17400395
start_block = 17400393
end_block = 17400395
timestamps_df = get_block_timestamps(start_block, end_block)


In [63]:
import pandas as pd

# Getting FLAIR data from csv 
flair = pd.read_csv("Range_LQTYETH_FLAIR_453456.csv")
flair = flair.drop(flair.columns[0], axis=1)

In [64]:
# Getting LVR data from csv
lvr = pd.read_csv("Range_LQTYETH_453456_LVR_17285193_17673993.csv")

In [65]:
import concurrent.futures

def apply_date(block_number):
  return get_date(get_timestamp(block_number))

def apply_timestamp(block_number):
  return get_timestamp(block_number)

with concurrent.futures.ThreadPoolExecutor() as executor:
    flair['timestamp'] = list(executor.map(apply_timestamp, flair['blockNumber']))
    flair['date'] = list(executor.map(apply_date, flair['blockNumber']))

In [66]:
# create a df called flair_daily which contains the last row of a given date within the flair df
flair_daily = flair.groupby('date').tail(1)
flair_daily

,flair,totalFee0,totalFee1,blockNumber,timestamp,date
13,0.000430,23.319363,0.016389,17287362,1684424135000,2023-05-18
65,0.002006,130.323882,0.061097,17294550,1684511663000,2023-05-19
94,0.002903,175.448816,0.098075,17300730,1684586783000,2023-05-20
121,0.003839,231.533463,0.130361,17308781,1684684727000,2023-05-21
173,0.005296,320.496328,0.180425,17315826,1684770587000,2023-05-22
204,0.005973,382.810323,0.188939,17322928,1684857047000,2023-05-23
244,0.007926,477.122030,0.273672,17329973,1684942547000,2023-05-24
268,0.009767,562.747468,0.355828,17336717,1685024495000,2023-05-25
305,0.019218,1075.712351,0.728329,17344211,1685115563000,2023-05-26
340,0.023275,1294.391690,0.888342,17351037,1685198327000,2023-05-27


In [67]:
# combine flair and lvr data on date
combined = pd.merge(flair_daily, lvr, on='date', how='inner')

In [68]:
# create a column called totalFee_USD
combined['totalFee_USD'] = combined['totalFee0'] * combined['token0_usd'] + combined['totalFee1'] * combined['token1_usd']

# drop timestamp, totalFee0, totalFee1
combined = combined.drop(['timestamp', 'totalFee0', 'totalFee1'], axis=1)
combined

,flair,blockNumber,date,block,daily_LVR_USD,cumulative_LVR_USD,token0_usd,token1_usd,in_range,totalFee_USD
0,0.000430,17287362,2023-05-18,17285193,103.671258,103.671258,1.340000,1826.78,True,61.187583
1,0.002006,17294550,2023-05-19,17292393,73.207763,175.781459,1.320000,1807.44,True,282.456318
2,0.002903,17300730,2023-05-20,17299593,67.810869,244.286725,1.310000,1814.58,True,407.803777
3,0.003839,17308781,2023-05-21,17306793,112.692400,357.066630,1.300000,1815.23,True,537.627858
4,0.005296,17315826,2023-05-22,17313993,78.572091,435.864933,1.290000,1816.38,True,741.160486
5,0.005973,17322928,2023-05-23,17321193,94.387406,539.706902,1.300000,1855.78,True,848.282917
6,0.007926,17329973,2023-05-24,17328393,72.014134,600.437032,1.250000,1816.98,True,1093.658983
7,0.009767,17336717,2023-05-25,17335593,86.990834,677.358786,1.240000,1786.51,True,1333.497609
8,0.019218,17344211,2023-05-26,17342793,54.853687,741.877057,1.250000,1812.00,True,2664.372631
9,0.023275,17351037,2023-05-27,17349993,67.838084,815.266936,1.280000,1825.56,True,3278.542527


In [69]:
# save combined df to csv
combined.to_csv('Range_LQTYETH_FLAIR_LVR_453456.csv')